In [1]:
import numpy as np 
import seaborn as sns
import pandas as pd 
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.express as px
import yfinance as yf
import quandl
from fredapi import Fred
sns.set(rc={'figure.figsize':(16,10)})
fred_key = "df4910b2cad947d95cf6ab16ba11d74d"
fred = Fred(api_key = fred_key)
quandl.ApiConfig.api_key = 'Qq5R29Xiqp2yUbb9dzNq'

In [2]:
df.shape

(60225, 12)

In [3]:
pd.set_option("display.max_rows",200)

In [4]:
code='RPI'
fred.get_series(code).to_frame(code)

,RPI
1959-01-01,2442.158
1959-02-01,2451.778
1959-03-01,2467.594
1959-04-01,2483.671
1959-05-01,2498.026
...,...
2022-10-01,17872.733
2022-11-01,17916.642
2022-12-01,17936.624
2023-01-01,17933.070


In [5]:
MacroCode=pd.read_csv(r'Data\Macro Variables.csv')

In [6]:
MacroCode.replace({'Average HOUST':'HOUST','S&P 500':'SP500'},inplace=True)

In [7]:
errors=[]
indicators=[]

for code in MacroCode['Variable']:
    if not ('S&P' in code):
        try:
            indicators.append(fred.get_series(code).to_frame(code))
        except ValueError:
            errors.append(code)
indicators.append(yf.download('^SP500-20')['Close'].to_frame('S&P: indust'))
indicators.append(quandl.get("MULTPL/SP500_DIV_YIELD_MONTH").squeeze().to_frame('S&P div yield'))
indicators.append(quandl.get("MULTPL/SP500_PE_RATIO_MONTH").squeeze().to_frame('S&P PE ratio'))


[*********************100%***********************]  1 of 1 completed


In [8]:
errors

[]

In [9]:
data=pd.concat(indicators,axis=1).fillna(method='ffill')

In [10]:
def deltaX(data):
    return data.diff()

def delta2X(data):
    return data-2*data.shift(1)+data.shift(2)

def logdata(data):
    return np.log(data)

def difflog(data):
    return np.log(data).diff()

def difflog2(data):
    return np.log(data)-2*np.log(data.shift(1))+np.log(data.shift(2))

def diffpercent(data):
    return (data/data.shift() - 1) - (data.shift()/data.shift(2) - 1)



transformation={1:(lambda x: x),
                2:deltaX,
                3:delta2X,
                4:logdata,
                5:difflog,
                6:difflog2,
                7:diffpercent,
                }

In [11]:
df_transformed=pd.DataFrame()

In [14]:
for column in data.columns:
    type=MacroCode[MacroCode['Variable'] == column]['Transformation'].iloc[0]
    df_transformed[column]=transformation[type](data[column])
    df_transformed=df_transformed.copy()

In [20]:
df_transformed.to_csv('transformed khnamaj.csv')


# some LSTM stuff

In [ ]:
import datetime

def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return datetime.datetime(year=year, month=month, day=day)

In [ ]:
import numpy as np

def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
  first_date = str_to_datetime(first_date_str)
  last_date  = str_to_datetime(last_date_str)

  target_date = first_date
  
  dates = []
  X, Y = [], []

  last_time = False
  while True:
    df_subset = dataframe.loc[:target_date].tail(n+1)
    
    if len(df_subset) != n+1:
      raise Exception(f'Error: Window of size {n} is too large for date {target_date}')

    values = df_subset['Close'].to_numpy()
    x, y = values[:-1], values[-1]

    dates.append(target_date)
    X.append(x)
    Y.append(y)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))
    
    if last_time:
      break
    
    target_date = next_date

    if target_date == last_date:
      last_time = True
    
  ret_df = pd.DataFrame({})
  ret_df['Target Date'] = dates
  
  X = np.array(X)
  for i in range(0, n):
    X[:, i]
    ret_df[f'Target-{n-i}'] = X[:, i]
  
  ret_df['Target'] = Y

  return ret_df

# Start day second time around: '2021-03-25'
windowed_df = df_to_windowed_df(df, 
                                '2021-03-25', 
                                '2022-03-23', 
                                n=3)
windowed_df